# Import

In [1]:
from pandas import read_csv
import numpy as np
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error

# Prepare Datasets

In [2]:
def get_fib_seq(n, scale_data=True):
    # Get the Fibonacci sequence
    seq = np.zeros(n)
    fib_n1 = 0.0
    fib_n = 1.0 
    for i in range(n):
            seq[i] = fib_n1 + fib_n
            fib_n1 = fib_n
            fib_n = seq[i] 
    scaler = []
    if scale_data:
        scaler = MinMaxScaler(feature_range=(0, 1))
        seq = np.reshape(seq, (n, 1))
        seq = scaler.fit_transform(seq).flatten()        
    return seq, scaler
 
fib_seq = get_fib_seq(10, False)[0]
print(fib_seq)

[ 1.  2.  3.  5.  8. 13. 21. 34. 55. 89.]


In [3]:
def get_fib_XY(total_fib_numbers, time_steps, train_percent, scale_data=True):
    dat, scaler = get_fib_seq(total_fib_numbers, scale_data)    
    Y_ind = np.arange(time_steps, len(dat), 1)
    Y = dat[Y_ind]
    rows_x = len(Y)
    X = dat[0:rows_x]
    for i in range(time_steps-1):
        temp = dat[i+1:rows_x+i+1]
        X = np.column_stack((X, temp))
    # random permutation with fixed seed   
    rand = np.random.RandomState(seed=13)
    idx = rand.permutation(rows_x)
    split = int(train_percent*rows_x)
    train_ind = idx[0:split]
    test_ind = idx[split:]
    trainX = X[train_ind]
    trainY = Y[train_ind]
    testX = X[test_ind]
    testY = Y[test_ind]
    trainX = np.reshape(trainX, (len(trainX), time_steps, 1))    
    testX = np.reshape(testX, (len(testX), time_steps, 1))
    return trainX, trainY, testX, testY, scaler
 
trainX, trainY, testX, testY, scaler = get_fib_XY(12, 3, 0.7, False)
print('trainX = ', trainX)
print('trainY = ', trainY)

trainX =  [[[ 8.]
  [13.]
  [21.]]

 [[ 5.]
  [ 8.]
  [13.]]

 [[ 2.]
  [ 3.]
  [ 5.]]

 [[13.]
  [21.]
  [34.]]

 [[21.]
  [34.]
  [55.]]

 [[34.]
  [55.]
  [89.]]]
trainY =  [ 34.  21.   8.  55.  89. 144.]


# Setting up the Network

In [10]:
# Set up parameters
time_steps = 20
hidden_units = 2
epochs = 10
 
# Create a traditional RNN network
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape, activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mse', optimizer='adam')
    return model
 
model_RNN = create_RNN(hidden_units=hidden_units, dense_units=1, input_shape=(time_steps,1), 
                   activation=['tanh', 'tanh'])
model_RNN.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 2)                 8         
                                                                 
 dense_2 (Dense)             (None, 1)                 3         
                                                                 
Total params: 11 (44.00 Byte)
Trainable params: 11 (44.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:

# Generate the dataset
trainX, trainY, testX, testY, scaler  = get_fib_XY(1200, time_steps, 0.7)
 
model_RNN.fit(trainX, trainY, epochs=epochs, batch_size=1, verbose=2)
 
 
# Evalute model
train_mse = model_RNN.evaluate(trainX, trainY)
test_mse = model_RNN.evaluate(testX, testY)
 
# Print error
print("Train set MSE = ", train_mse)
print("Test set MSE = ", test_mse)

Epoch 1/10
826/826 - 7s - loss: 0.0024 - 7s/epoch - 8ms/step
Epoch 2/10
826/826 - 5s - loss: 0.0021 - 5s/epoch - 6ms/step
Epoch 3/10
826/826 - 5s - loss: 0.0019 - 5s/epoch - 6ms/step
Epoch 4/10
826/826 - 5s - loss: 0.0017 - 5s/epoch - 6ms/step
Epoch 5/10
826/826 - 5s - loss: 0.0015 - 5s/epoch - 6ms/step
Epoch 6/10
826/826 - 5s - loss: 0.0013 - 5s/epoch - 6ms/step
Epoch 7/10
826/826 - 5s - loss: 0.0012 - 5s/epoch - 6ms/step
Epoch 8/10
826/826 - 5s - loss: 0.0010 - 5s/epoch - 6ms/step
Epoch 9/10
826/826 - 5s - loss: 9.0143e-04 - 5s/epoch - 6ms/step
Epoch 10/10
826/826 - 5s - loss: 8.0734e-04 - 5s/epoch - 6ms/step
12/12 [==============================] - 0s 4ms/step - loss: 5.9310e-04
Train set MSE =  0.0006957798614166677
Test set MSE =  0.0005930991610512137


# Adding a Custom Attention Layer to the Network

In [12]:
# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)
 
    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

# RNN Network with Attention Layer

In [13]:
def create_RNN_with_attention(hidden_units, dense_units, input_shape, activation):
    x=Input(shape=input_shape)
    RNN_layer = SimpleRNN(hidden_units, return_sequences=True, activation=activation)(x)
    attention_layer = attention()(RNN_layer)
    outputs=Dense(dense_units, trainable=True, activation=activation)(attention_layer)
    model=Model(x,outputs)
    model.compile(loss='mse', optimizer='adam')    
    return model    
 
model_attention = create_RNN_with_attention(hidden_units=hidden_units, dense_units=1, 
                                  input_shape=(time_steps,1), activation='tanh')
model_attention.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20, 1)]           0         
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 20, 2)             8         
                                                                 
 attention_1 (attention)     (None, 2)                 22        
                                                                 
 dense_3 (Dense)             (None, 1)                 3         
                                                                 
Total params: 33 (132.00 Byte)
Trainable params: 33 (132.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Train and Evaluate the Deep Learning Network with Attention

In [14]:
model_attention.fit(trainX, trainY, epochs=epochs, batch_size=1, verbose=2)
 
# Evalute model
train_mse_attn = model_attention.evaluate(trainX, trainY)
test_mse_attn = model_attention.evaluate(testX, testY)
 
# Print error
print("Train set MSE with attention = ", train_mse_attn)
print("Test set MSE with attention = ", test_mse_attn)

Epoch 1/10
826/826 - 8s - loss: 0.0015 - 8s/epoch - 9ms/step
Epoch 2/10
826/826 - 5s - loss: 0.0015 - 5s/epoch - 6ms/step
Epoch 3/10
826/826 - 5s - loss: 0.0014 - 5s/epoch - 6ms/step
Epoch 4/10
826/826 - 5s - loss: 0.0014 - 5s/epoch - 6ms/step
Epoch 5/10
826/826 - 5s - loss: 0.0014 - 5s/epoch - 6ms/step
Epoch 6/10
826/826 - 6s - loss: 0.0013 - 6s/epoch - 7ms/step
Epoch 7/10
826/826 - 5s - loss: 0.0012 - 5s/epoch - 6ms/step
Epoch 8/10
826/826 - 5s - loss: 0.0012 - 5s/epoch - 6ms/step
Epoch 9/10
826/826 - 5s - loss: 0.0011 - 5s/epoch - 6ms/step
Epoch 10/10
826/826 - 5s - loss: 0.0010 - 5s/epoch - 6ms/step
12/12 [==============================] - 0s 6ms/step - loss: 7.3636e-04
Train set MSE with attention =  0.0009076849091798067
Test set MSE with attention =  0.0007363584591075778
